In [9]:
from typing import Annotated
from typing_extensions import TypedDict, Optional

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import add_messages
from langchain_deepseek import ChatDeepSeek
from langchain_ollama import ChatOllama

from pydantic import BaseModel, Field

llm = ChatOllama(model="qwen3:4b")

message = [HumanMessage(content="Hello, world!")]

llm.invoke(message)

AIMessage(content='<think>\nOkay, the user sent "Hello, world!" which is a common greeting. I need to respond appropriately. Since it\'s a simple message, I should acknowledge it and maybe add a friendly reply. Let me check if there\'s any specific context I\'m missing. No, it\'s just a greeting. I\'ll respond with a cheerful message to keep the conversation going. Maybe something like "Hello! How can I assist you today?" That sounds good. It\'s polite and opens the door for further interaction. I should make sure to keep it friendly and welcoming. Alright, that\'s ready to send.\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-06-22T11:48:32.707954634Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3191429614, 'load_duration': 1951281808, 'prompt_eval_count': 12, 'prompt_eval_duration': 151085840, 'eval_count': 137, 'eval_duration': 1088446800, 'message': Message(role='assistant', content=''

In [25]:


class userinfo(BaseModel):
    name: str = Field(..., description="The name of the user")
    id: str = Field(..., description="The id of the user, 身份证号")
    phone: Optional[str] = Field(None, description="The phone number of the user")
    introduction: Optional[str] = Field(None, description="A brief introduction of the user")


structurt = llm.with_structured_output(userinfo)

# answer = structurt.invoke("My name is John Doe, my id is 123456789012345678, my phone number is 123-456-7890, and I am a software engineer.")


class input_state(TypedDict):
    mm: Annotated[list[str], add_messages]
    # mm: int


def node_a(input:input_state):
    """A node that processes the input state."""
    print(input)
    x = input["mm"][-1]
    return {"mm": ['10']}

def node_b(input:input_state):
    """A node that processes the input state."""
    x = input["mm"][-1]
    print(input)
    return {"mm": ['84']}

graph = StateGraph(input_state)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)

graph.add_edge(START, "node_a")
graph.add_edge("node_a", "node_b")
graph.add_edge("node_b", END)

graph = graph.compile()

answer = graph.invoke({'mm': ['23']})
print(answer)

{'mm': [HumanMessage(content='23', additional_kwargs={}, response_metadata={}, id='c1e24fa3-124a-4084-bc63-1685e6ae6b83')]}
{'mm': [HumanMessage(content='23', additional_kwargs={}, response_metadata={}, id='c1e24fa3-124a-4084-bc63-1685e6ae6b83'), HumanMessage(content='10', additional_kwargs={}, response_metadata={}, id='8877211c-aa01-4778-b831-628b4b7c3523')]}
{'mm': [HumanMessage(content='23', additional_kwargs={}, response_metadata={}, id='c1e24fa3-124a-4084-bc63-1685e6ae6b83'), HumanMessage(content='10', additional_kwargs={}, response_metadata={}, id='8877211c-aa01-4778-b831-628b4b7c3523'), HumanMessage(content='84', additional_kwargs={}, response_metadata={}, id='32861bd6-ac86-4147-a5e4-51e5f40e1877')]}


In [28]:
class input_state(TypedDict):
    mm: Annotated[list[AnyMessage], add_messages]
    # mm: int


def node_a(input:input_state):
    """A node that processes the input state."""
    prompt = "你是一位网络梗专家，专门帮忙解答网络梗相关的问题。"
    message = [{"role": "system", "content": prompt}]
    return {"mm": message}

def node_b(input:input_state):
    """A node that processes the input state."""
    messages = input["mm"] + [{"role": "user", "content": "对于所有回答要翻译成英文、中文、日语三种语言"}]
    answer = llm.invoke(messages)
    print(messages)

    return {"mm": answer}

graph = StateGraph(input_state)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)

graph.add_edge(START, "node_a")
graph.add_edge("node_a", "node_b")
graph.add_edge("node_b", END)

graph = graph.compile()

answer = graph.invoke({"mm": [HumanMessage(content="曼波、曼波")]})
print(answer)
print(answer["mm"][-1].content)

[HumanMessage(content='曼波、曼波', additional_kwargs={}, response_metadata={}, id='b835ee54-e0cb-4a50-a781-4bee4fe88a5d'), SystemMessage(content='你是一位网络梗专家，专门帮忙解答网络梗相关的问题。', additional_kwargs={}, response_metadata={}, id='a423b4be-0cf0-4b80-9b39-35b1334161d5'), {'role': 'user', 'content': '对于所有回答要翻译成英文、中文、日语三种语言'}]
{'mm': [HumanMessage(content='曼波、曼波', additional_kwargs={}, response_metadata={}, id='b835ee54-e0cb-4a50-a781-4bee4fe88a5d'), SystemMessage(content='你是一位网络梗专家，专门帮忙解答网络梗相关的问题。', additional_kwargs={}, response_metadata={}, id='a423b4be-0cf0-4b80-9b39-35b1334161d5'), AIMessage(content='<think>\n好的，用户发来了“曼波、曼波”，然后要求所有回答翻译成英文、中文、日语三种语言。首先，我需要确认“曼波”指的是什么。可能是指舞蹈“曼波舞”（Mambo），这是一种源自古巴的舞蹈，节奏欢快，动作灵活。也有可能是其他含义，但根据常见网络用语，曼波舞应该是主要的。\n\n接下来，用户可能是在测试我的知识库，或者想了解曼波舞的相关信息。需要确保回答准确，同时满足三种语言的要求。另外，用户可能对曼波舞有特别的兴趣，或者想用在某个场合，比如社交、教学等。\n\n需要检查是否有其他可能的解释，但根据常见用法，曼波舞是正确的。然后，翻译部分要准确，英文是Mambo，中文是曼波舞，日语是マンボ。然后，回答部分需要涵盖曼波舞的起源、特点、流行情况等，确保信息正确，同时用三种语言表达。\n\n另外，用户可能希望得到简洁明了的回答，所以需要分点说明，但用户没有要求分点，所以保持段落形式即可。确保翻译无